Install libraries

In [1]:
!pip install gensim --upgrade -q

     |████████████████████████████████| 24.1 MB 1.9 MB/s 


In [2]:
import gensim
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from operator import itemgetter
import tqdm
import nltk
import re
import multiprocessing

In [3]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Clean dataset

In [4]:
def preprocess_text(sentence):
    # Lowercase
    sentence = sentence.lower()
    
    # Remove all non-alphabets (punctuation, numbers, new-line characters and extra-spaces)
    sentence = re.sub(r'[^a-zA-Z]+', ' ', sentence)
    sentence = sentence.replace('\n', '')
    # sentence = re.sub('\s\s+', ' ', sentence)
    
    # Tokenize & remove stop-words
    word_list = nltk.word_tokenize(sentence)    
    stopwords_list = set(nltk.corpus.stopwords.words('english'))
    word_list = [word for word in word_list if word not in stopwords_list]
    
    # Remove very small words, length < 3, they don't contribute any useful information
    word_list = [word for word in word_list if len(word) > 3]
        
    # Stem & Lemmatize
    porter_stemmer = nltk.stem.PorterStemmer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    word_list = [porter_stemmer.stem(word) for word in word_list]
    word_list = [lemmatizer.lemmatize(word) for word in word_list]
    
    sentence = ' '.join(word_list)
    
    return sentence

Load dataset

In [5]:
from sklearn.datasets import fetch_20newsgroups
news_group = fetch_20newsgroups(subset='train')

news_group_data = news_group.data
news_group_target_names = news_group.target_names
news_group_target = news_group.target

# Creating a dataframe from the loaded data
news_df = pd.DataFrame({'news': news_group_data, 
                        'class': news_group_target})

news_df.reset_index(drop=True, inplace= True)

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(news_df, test_size=0.2)

Pre-process dataset

In [7]:
# Preprocess the news description
tqdm.tqdm.pandas()
train_df['news_tokenized'] = train_df['news'].progress_apply(lambda x: preprocess_text(str(x)))


100%|██████████| 9051/9051 [00:55<00:00, 162.97it/s]


Bag of Words

In [8]:
train_documents = train_df['news_tokenized'].str.split()
dictionary = gensim.corpora.Dictionary(train_documents)
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=20000)

train_corpus = [dictionary.doc2bow(document) for document in train_documents]

Create dataframe

In [9]:
performance_metrics = pd.DataFrame(columns=['feature-extraction','run#', 'epoch', 'state', 'AMI','ARI','NMI'])


NMF Epochs Grid search

In [14]:
for run in range(1, 3, 1):
    print('Run #', run)
    for epoch, state in zip(range(25, 250, 25), range(2, 42, 2)):
        pred_labels = []
        gensim_nmf = gensim.models.Nmf(corpus=train_corpus, num_topics=20, id2word=dictionary, chunksize=1000, passes=epoch, eval_every=10, minimum_probability=0, 
                                   random_state=state, kappa=1)
        
        for train_doc in train_corpus:
            pred_label = max(gensim_nmf[train_doc], key=itemgetter(1))[0]
            pred_labels.append(pred_label)
        
        ami = metrics.adjusted_mutual_info_score(train_df['class'], pred_labels)
        ari = metrics.adjusted_rand_score(train_df['class'], pred_labels)
        nmi = metrics.normalized_mutual_info_score(train_df['class'], pred_labels)

        print(run, epoch, state)
        
        performance_metrics = performance_metrics.append({'feature-extraction':'Bag of Words', 'run#':run, 'epoch':epoch, 'state':state, 'AMI':'{:f}'.format(ami), 'ARI':'{:f}'.format(ari), 'NMI':'{:f}'.format(nmi)}, ignore_index=True)
        

Run # 1
1 25 2
1 50 4
1 75 6
1 100 8
1 125 10
1 150 12
1 175 14
1 200 16
1 225 18
Run # 2
2 25 2
2 50 4
2 75 6
2 100 8
2 125 10
2 150 12
2 175 14
2 200 16
2 225 18


Save metrics

In [15]:
performance_metrics.to_csv('/content/drive/MyDrive/Big Data/Code/Metrics/performance-bow_nmf.csv', index=False)
performance_metrics[['run#','epoch','state','AMI','ARI','NMI']] = performance_metrics[['run#','epoch','state','AMI','ARI','NMI']].apply(pd.to_numeric, errors='coerce')
performance_metrics.dtypes
mean_performance = performance_metrics.groupby('epoch', as_index=False)[['AMI','ARI','NMI']].mean()
mean_performance.to_csv('/content/drive/MyDrive/Big Data/Code/Metrics/mean_performance-bow_nmf.csv', index=False)